In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import glob

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Сентиментальный анализ

Вынесем общие функции

In [3]:
import nltk

nltk.download([
    "vader_lexicon"
])

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

Функция самовызова для сентиментального анализа по VADER

In [43]:
from nltk.sentiment import SentimentIntensityAnalyzer

def sentimently_analyze_vader(sentence):
    sia = SentimentIntensityAnalyzer()
    polarity_scores = sia.polarity_scores(str(sentence))
    return polarity_scores

Функция самовызова для сентиментального анализа по TextBlob

In [5]:
from textblob import TextBlob

MOOD_THRESHOLD = 0.3

def sentimently_analyze_textblob(sentence):
    res = TextBlob(str(sentence))
    print("TextBlob")
    print(res)
    result = "neg"
    if res.sentiment.polarity > MOOD_THRESHOLD:
        result = "pos"

    return result

Библиотека Pysentiment для сентиментального анализа по словарям Harvard-IV и Loughran & Mcdonald

In [6]:
!pip install pysentiment2

     |████████████████████████████████| 1.9 MB 5.2 MB/s 


Функция самовызова для сентиментального анализа по Harvard-IV

In [73]:
import pysentiment2 as ps

def sentimently_analyze_harvard(sentence):
    hiv4 = ps.HIV4()
    
    tokens = hiv4.tokenize(str(sentence))
    res = hiv4.get_score(tokens)
    res['word_count'] = len(tokens)
    return res

Функция самовызова для сентиментального анализа по Loughran & Mcdonald

In [74]:
from nltk.collections import LazyMap
import pysentiment2 as ps

def sentimently_analyze_mcdonald(sentence):
    lm = ps.LM()
    
    tokens = lm.tokenize(str(sentence))
    res = lm.get_score(tokens)
    res['word_count'] = len(tokens)
    return res

Загружаем свечи и новости по каждой компании и проводим сентиментальный анализ

In [47]:
news_file_path = '/content/drive/My Drive/Colab Notebooks/Apple/'
name_file = "AAPL_part1"

In [75]:
def merge_sentiment_analysis(df_news, scores, metrics_name):
  # Convert the 'scores' list of dicts into a DataFrame
  scores_df = pd.DataFrame(scores.apply(pd.Series))
  for column_name in scores_df.columns:
    scores_df.rename(columns={column_name: f"{column_name}_{metrics_name}"}, inplace=True)

  # Join the DataFrames of the news and the list of dicts
  df_news = df_news.join(scores_df, rsuffix='_right')
  return df_news

In [82]:
df_news = pd.read_csv(f'{news_file_path}{name_file}.csv', index_col=0)
df_news['title'] = df_news['title'].astype('str') 
df_news['description'] =  df_news['description'].astype('str') 

# подсчитаем общее кол-во слов в заголовке и описании
df_news['word_count_title'] = df_news['title'].apply(lambda x: len(x.split())) 
df_news['word_count_description'] =  df_news['description'].apply(lambda x: len(x.split())) 

# запуск сентиментального анализа
vader = df_news['title'].apply(sentimently_analyze_vader)
df_news = merge_sentiment_analysis(df_news, vader, "vader_title")

vader = df_news['description'].apply(sentimently_analyze_vader)
df_news = merge_sentiment_analysis(df_news, vader, "vader_desc")

harvard = df_news['title'].apply(sentimently_analyze_harvard)
df_news = merge_sentiment_analysis(df_news, harvard, "harvard_title")

harvard = df_news['description'].apply(sentimently_analyze_harvard)
df_news = merge_sentiment_analysis(df_news, harvard, "harvard_desc")

mcdonald = df_news['title'].apply(sentimently_analyze_mcdonald)
df_news = merge_sentiment_analysis(df_news, mcdonald, "mcdonald_title")

mcdonald = df_news['description'].apply(sentimently_analyze_mcdonald)
df_news = merge_sentiment_analysis(df_news, mcdonald, "mcdonald_desc")

df_news.to_csv(f"{news_file_path}{name_file}_result.csv", encoding='utf-8', index=False)